<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/Rola_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考文章：https://note.com/npaka/n/ndb287a48b682

In [2]:
%%script false --no-raise-error
from google.colab import drive
drive.mount('/content/drive')

In [3]:
%cd '/content/drive/MyDrive/ML_Practices/Lora_test_1'
!pwd

/content/drive/MyDrive/ML_Practices/Lora_test_1
/content/drive/MyDrive/ML_Practices/Lora_test_1


In [ ]:
#download Lora
%cd  /content/drive/MyDrive/StableDifussion/Lora
!git clone https://github.com/cloneofsimo/lora.git

!pip install accelerate bitsandbytes
!pip install -r requirements.txt
#%cd lora
#!pip install .
!pwd

[Errno 2] No such file or directory: '/content/drive/MyDrive/StableDifussion/Lora'
/content/drive/MyDrive/ML_Practices/Lora_test_1
Cloning into 'lora'...
remote: Enumerating objects: 934, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 934 (delta 346), reused 301 (delta 301), pack-reused 527
Receiving objects: 100% (934/934), 182.96 MiB | 26.44 MiB/s, done.
Resolving deltas: 100% (560/560), done.
Updating files: 100% (60/60), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 19.5 MB/s eta 0:00:00


参考文章：https://torch.classcat.com/2023/04/12/blog-sd-webui-colab-lora-training-by-diffusers/

In [ ]:
!pip install diffusers==0.14.0 transformers accelerate safetensors

In [ ]:
!git clone https://github.com/huggingface/diffusers.git -b v0.14.0

In [ ]:
from accelerate.utils import write_basic_config

write_basic_config(mixed_precision="fp16", save_location="/content/lora/default_config.yaml")
!cat /content/lora/default_config.yaml

In [ ]:
!cat /content/lora/default_config.yaml

In [ ]:
%%time

!accelerate launch --config_file="/content/lora/default_config.yaml" \
  /content/drive/MyDrive/ML_Practices/Lora_test_1/diffusers/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"  \
  --instance_data_dir="/content/drive/MyDrive/ML_Practices/Lora_test_1/resized_sample_data_3" \
  --output_dir="/content/lora/output" \
  --instance_prompt="shs" \
  --resolution=512 \
  --train_batch_size=1 \
  --sample_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=200 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=3000 \
  --seed="0"

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler

pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
    ).to('cuda')

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

In [ ]:
pipe.unet.load_attn_procs("/content/lora/output/pytorch_lora_weights.bin")

In [ ]:
#generator = [torch.Generator(device="cpu").manual_seed(i) for i in range(4)]

images = pipe(prompt=" (((shs))),1 girl,cute ,simple,happy,computer,business,white background,pure background,sticker",
              negative_prompt="authentic,real people,complicated,grey",
              #generator=generator,
              num_inference_steps=30,
              height=512,
              width=512,
              num_images_per_prompt=4,
              guidance_scale=7.0).images

image_grid(images, 2, 2)